In [ ]:
import pandas as pd

file_name=[
    '01','02','03','04','05','06','07','08','09','10',
    '11','12','13','14','15','16','17','18','19','20',
    '21','22','23','24','25','26','27','28','29','30',
    '31','32','33','34','35','36','37','38','39','40',
    '41','42','43','44','45','46','47','48','49','50',
    '51','52','53','54','55','56','57','58','59','60']

data=[]

for i in range(len(file_name)):
    data.append(pd.read_csv('../data/'+ file_name[i] +'_tracks.csv'))

# Standard deviation of speed

# Create Pull Request


In [ ]:
# add standard deviation and add into the dataframe as a new column
import numpy as np
for i in range(len(file_name)):
    speed_len=len(data[i]['xVelocity'])
    data[i]['DV1']=np.sqrt((data[i]['xVelocity']-data[i]['xVelocity'].mean())**2/speed_len)

print(data[0]['DV1'])

# DV1 


In [ ]:
# calculate Standard deviation of longitudinal deceleration or acceleration
import numpy as np
for i in range(len(file_name)):
    long_acc=data[i]['xAcceleration'][data[i]['xAcceleration']>0] # longitudinal acceleration
    long_len=len(long_acc)
    data[i]['DV2']=np.sqrt((long_acc-long_acc.mean())**2/long_len)
    
print(data[40]['DV2'])

In [ ]:
# calculate Mean absolute deviation of speed

for i in range(len(file_name)):
    speed_len=len(data[i]['xVelocity'])
    data[i]['DV6']=np.abs((data[i]['xVelocity']-data[i]['xVelocity'].mean())**2)/speed_len

print(data[0]['DV6'])

In [ ]:

# import DV1 ,DV2 ,DV6 as new csv files 


for i in range(len(file_name)):
    DV1=data[i]['DV1']
    DV2=data[i]['DV2']
    DV6=data[i]['DV6']
    header=['DV1','DV2','DV6']
    data[i].to_csv('../data_new/'+ file_name[i] +'_tracks_new.csv',index=False,columns=header)
    

In [ ]:
DV1=data[0][['DV1','DV2','DV6']].groupby([data[0]['id'],data[0]['DV1'],data[0]['DV2'],data[0]['DV6']],dropna=False).mean()
new =pd.DataFrame(DV1).groupby(level=0).mean()
new.fillna(0,inplace=True)
#data[0][data[0]['id']==2]['DV6'].mean()

new.to_csv('../data_new/new.csv')